In [14]:
#!pip install langchain faiss-cpu sentence-transformers huggingface-hub transformers langchain-community llama-cpp-python

In [15]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("test2.txt",encoding="utf-8")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)
len(chunks)

7

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    encode_kwargs={"normalize_embeddings": True}
)


db = FAISS.from_documents(chunks, embeddings)
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 5,
        "score_threshold": 0.6
    }
)



'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8dd7e868-8a25-4b77-916a-334bb635c297)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


In [17]:
from langchain_community.llms import LlamaCpp


llm = LlamaCpp(
    model_path=r"D:\final Year project\Research_backend\Google_search\model\Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    temperature=0.2,
    max_tokens=256,
    n_ctx=4096,
    verbose=False
)


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [18]:
def rag_chain(question):
    # 1. Retrieve similar docs
    docs = retriever.invoke(question)

    if not docs:
        return "நீங்கள் கேட்ட கேள்விக்கான பதில், இங்கு உள்ள தகவல்களில் இல்லை. இணையத்தில் தேடி பதில் சொல்ல வேண்டுமா?"


    # 2. Combine the doc text
    context = "\n\n".join([d.page_content for d in docs])

    # 3. Format the prompt
    prompt_text = f"""

கீழே வழங்கப்பட்டுள்ள 'செய்தி' தவிர வேறு எந்த அறிவையும் பயன்படுத்த வேண்டாம். 
கேள்விக்கான பதில் 'செய்தி' இல் இல்லை என்றால், 'எனக்கு தெரியவில்லை' என்று மட்டும் கூறவும்.

செய்தி:
{context}

கேள்வி: {question}

"""

    # 4. Call the model
    raw = llm.invoke(prompt_text)
    answer = raw.replace(prompt_text, "").strip()
    print(prompt_text)
    return answer

In [19]:
response = rag_chain("நகரமயமாக்கல் வேகமாக வளர்வதற்கான முக்கிய காரணம் என்ன?")
print(response)





கீழே வழங்கப்பட்டுள்ள 'செய்தி' தவிர வேறு எந்த அறிவையும் பயன்படுத்த வேண்டாம். 
கேள்விக்கான பதில் 'செய்தி' இல் இல்லை என்றால், 'எனக்கு தெரியவில்லை' என்று மட்டும் கூறவும்.

செய்தி:
சமீப காலங்களில் நகரமயமாக்கல் வேகமாக வளர்ந்து வருகின்றது. இதன் காரணமாக கிராமப்புறங்களில் இருந்து நகரங்களுக்கு வேலை வாய்ப்புகளை தேடி மக்கள் இடம்பெயர்ந்து வருகின்றனர். நகரங்களில் தொழிற்சாலைகள், தகவல் தொழில்நுட்ப நிறுவனங்கள் மற்றும் சேவைத் துறைகள் அதிகரித்துள்ளதால் வேலை வாய்ப்புகள் அதிகமாக உள்ளன. ஆனால்

சுற்றுச்சூழல் சமநிலை பாதிக்கப்படுகிறது. இதனை சமாளிக்க அரசு தரப்பில் இருந்து நிலையான நகரமயமாக்கல் (sustainable urbanization) திட்டங்கள் அறிமுகப்படுத்தப்பட்டு வருகின்றன. பொதுப் போக்குவரத்தை மேம்படுத்துதல், மழைநீர் சேகரிப்பு, பசுமை பூங்காக்கள் அமைத்தல் போன்ற நடவடிக்கைகள் மூலம் நகர வாழ்க்கையின் தரத்தை

போன்ற நடவடிக்கைகள் மூலம் நகர வாழ்க்கையின் தரத்தை உயர்த்த முயற்சிகள் மேற்கொள்ளப்பட்டு வருகின்றன.

வேலை வாய்ப்புகள் அதிகமாக உள்ளன. ஆனால் இதன் மறுபக்கம், போக்குவரத்து நெரிசல், காற்று மாசுபாடு, குடிநீர் பற்றாக்குறை போன்ற பல ப

In [20]:
response = rag_chain("தமிழ்நாட்டின் தலைநகர் எது?")
print(response)

No relevant docs were retrieved using the relevance score threshold 0.6


நீங்கள் கேட்ட கேள்விக்கான பதில், இங்கு உள்ள தகவல்களில் இல்லை. இணையத்தில் தேடி பதில் சொல்ல வேண்டுமா?
